# Cleaning Engin X data

## Introduction 

Engin X is an instrument that uses neutron spectroscopy to find various characteristics of a material most notably the strain/stress. Over the years as scientists have used Engin X a large backlog of data has accumulated, with inconsistencies; format, variable names, ect. This is an attempted to “Clean the data” by formatting it in a clearer and more consistent fashion. 

## Method 

Firstly the tools I have used are python with various libraries; os, numpy, math, xlsxwriter, pandas for most of the data manipulation, threading to speed up the code as the data set was quite large and I had a server to work with and mantid and ISIS specific module that gave me access to data that was in .raw and .nsx files types. 

I define “Cleaning the data” as moving it from the format it was in: txt files titled with a variable name and unique run number and consisted of a column of time and a column of a measured variable, these files were separated by their cycles, all together Engin X has ran for about 78 cycles. I moved all this data to a new format focusing on it being pandas friendly, the data is now in csv files for each cycle with all the data for each cycle in one large csv file that can be pulled into a pandas dataframe using pd.read_csv(using chunksize is recommended). Data from the neutron detector is in a separate file that can be accessed using the keys in the large csv file and a function I have made. This all in all has made the data far easier to work with and sort through using pandas’s pandas.dataframe.groupby() function. 

## Implementation

I “Cleaned the data” in steps as it makes it easier to understand and allowed me to compartmentalise the problems. I go over Threading as the last step.

In [2]:
import os
import math
import xlsxwriter
import numpy as np
import pandas as pd
from pandas import isnull

### Step 1

As mentioned earlier the data in the .raw and .nsx files were extracted using Mantid, they both have the same data contained within them, however .nsx files were only recently used so I extracted all the data using .raw files as it was more universally used. Mantid hasn’t been fully updated to Python 3 yet, to get around this I used Mantid for python 2.7 for the first bit of this project. I extracted 7 pieces of data from the raw files, the X and Y data from the North and South detectors, neutron beam start time, neutron beam end time, title of the run. I then took this data and put it in a more accessible file type, txt with a separator string that never shows up in the data so I could easily split up the data later using python’s split method on strings. This is all done in the file titled Mantid in the repository.   

### Step 2

Throughout the code I use many functions to manipulate the original data as the indexing changed over time and I needed to do things like find the delimiter of the run number or the index of all the runs in a given cycle. Thus I made the following functions.

In [ ]:
def CreateFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

def FindDel(S):
    """This function takes a the first string in a list,(S) that starts with ENG, it finds the nomancature 
    the file uses for the begining of each file, e.g ENG44007 or ENGINX00305258, it shall also work if more numbers
    are added or the name changes so long as ENG followed by anything other than "." or "_" or " " is still 
    the start of each file"""
    for i in S:
        if i[0:3] == 'ENG':
            n = 1
            for j in i:
                if j == '.':
                    break
                elif j == ' ':
                    break
                elif j == '_':
                    break
                else: 
                    n += 1 
            return n
        else:
            continue


def InitialRun(S):
    """This takes S a list of filenames and returns the first run index"""
    last_int_index = FindDel(S) -1
    first_int_index = 0 
    for i in S:                                                          
        if i[0:3] == 'ENG':
            for j in range(last_int_index):
                try:
                    int(i[j])
                    run_number = i[first_int_index:last_int_index]
                    break
                except:
                    first_int_index += 1
        else:
            continue
        break
    return((run_number))

def CycleRuns(S):
    """This takes a list of filenames and returns a list of all the runs in the file"""
    Start_run = int(InitialRun(S))
    Last_run = int(InitialRun(reversed(S)))
    Runs = range(Start_run,Last_run)
    return (list(Runs))

def RunIndex(s,n): #ENGINX00305258
    """s is a string in the for m of a filename from the enginx data and returns it's run number, n is the FindDel"""
    last_int_index = n-1
    first_int_index = 0
    for j in range(last_int_index):
        try:
            int(s[j])
            return s[first_int_index:last_int_index]
        except:
            first_int_index += 1

### Step 3

As all the same variable in the data over time has been given many different names I needed a way to find a way to identify each variable to a single name. I chose that this would be best done with a dictionary that takes every given variable name(Names that are given in the uncleaned data) as keys and returns the universal variable name(Names that I have given to the variables) as values. The universal names are also used as the headers for the csv files once the data is cleaned. 

This was done in two parts first I used the code below to make a spreadsheet of all the unique non temporary file names and the cycle that file was found. Temporary files have data that are in the .raw files, they are created so scientists can check how the data is coming along mid experiment. 

In [ ]:
directory = "D:\\" # Directory where data is being stored
array_names = []
file_years = []
long_array_names = []
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[n:] in array_names: #stop repeated file names from entering 
                pass
            elif len(filename) < n+1: # Stop useless file names from entering (e.g copy)
                pass
            elif filename[n] in ("S", "s", "N", "n"):# Stop temporary files from entering (e.g .s01 or .n001)
                try:
                    int(filename[n+1]) # all temp files are 's' or 'n' followed by a int
                    temp_files.append(filename)
                except ValueError: 
                    array_names.append(filename[n:])
                    long_array_names.append(filename)
                    file_years.append(foldername)
            else: # most unique file names go here 
                array_names.append(filename[n:])
                long_array_names.append(filename)
                file_years.append(foldername)
                
#print(file_years)
#print(array_names)
#print(long_array_names)

workbook   = xlsxwriter.Workbook('Filenames.xlsx')

worksheet1 = workbook.add_worksheet()
worksheet2 = workbook.add_worksheet()

worksheet1.write_column('A1', array_names)
worksheet2.write_column('A1', long_array_names)
worksheet1.write_column('B1', file_years)
worksheet2.write_column('B1', file_years)
workbook.close()

From here the only way I thought of writing in the universal names is by manually typing them into the spreadsheet, which is what I did. I made a function that would take the spreadsheet and turn it into a dictionary and give a list of the column names. 

In [ ]:
path = "C:\\Users\\zahaab\\ENGIN_X_DATA_CLEANING\\File_names_2013-19.xlsx" # The spreadsheet of file names
def Translator(path):
    """This function takes the path to a spreadsheet and makes a dictionary between two columns and returns the 
    universal file names as a list, I use it to make a dictionary to translate the given file names to the universal 
    names"""
    xls = pd.ExcelFile(path)
    df = xls.parse(sheet_name = "Sheet1")
    df = df[["Filename", "Column_name"]]
    df = df.fillna(0)
    true_name_translater = dict(zip(df["Filename"],df["Column_name"]))
    s = set(true_name_translater.values())
    col_names = list(s)
    col_names.remove(0)
    col_names.append("measured_temperature_U")#This is used later as some temperature mesurments don't 
    #note the number of the input used
    col_names.append("unregistered_value") #A place for values that aren't specified
    return true_name_translater, col_names

true_name_translater, col_names = Translator(path)

### Step 4

This is the largest step and it involves extracting all the data from the txt files and the .raw files and then putting those into a new format. These are now in directories with the name of the cycle, those directories consists of txt files, named after the unique run number of the data that it contains, the txt files are in csv format with all the data from that run in one place. This is done using 4 functions that I’ll explain one at a time. 

There are 3 data types being passed back and forth throughout this step:

raw_data: dictionary of the key’s being the from the strings from the neutron data list shown later and the values being the data itself.

time_list: is a numpy array of all the time values in a given run

Run_data: this is the data from all the .txt values held in a list of two element tuples; element 1 is the universal variable name, element 2 is a dictionary of time\date keys and float values of the variable being measured at that time. Reasoning for this is that it’s faster than a dictionary in a dictionary and lets the NewRun function be far more concise than other data structures. 

In [ ]:
def AddRawData(run_number):
    """Takes the run number of a raw file, this returns  a dict of the .raw data that came from that raw file as a 
    dictionary of the key’s being the from the strings from the neutron data list shown later and the values being 
    the data itself. However for the X and Y values for the North and South detectors it inputs a string into the 
    dictionary and makes a file in a different directory with the data, this string can be used 
    with the NeutronDataTranslator function to get the Neutron data."""
    file_path = output+"\\"+str(run_number)+".txt"
    output_file = open(file_path, "r")
    contents = output_file.read()
    split_contents = contents.split("kamehameha") # Had to pick somthing that wouldn't be in data
    del split_contents[-1]
    clean_split_contents = []
    for i,j in zip(split_contents, data):
        a = i.replace("\n","").replace("[","").replace("]","")
        a = a.split(" ")
        a = list(filter(None, a))
        if "data" in j:
            clean_split_contents.append(j[0:16]+str(run_number))
            Neutron_file = open(Neutron_folder+"\\"+j[0:16]+str(run_number)+".txt", "w+")
            Neutron_file.write(str(a))
            Neutron_file.close()
        elif "time" in j:
            untdate = i.replace('T',' ')
            clean_split_contents.append(untdate)
        else:
            clean_split_contents.append(a)
    raw_data = dict(zip(data, list(clean_split_contents)))
    output_file.close()
    os.remove(file_path) 
    return raw_data

As mentioned in step one the .raw data is now in txt files with a string to split on. The AddRawData function opens the txt file with a certain run’s raw data splits it as to get the different data seperated. An issue you will notice that comes up a lot is that some of the time values in the data have a “T” in the center and some don’t, this is often solved in the data with an if statement which checks if there is a “T” and if one is found removes it and reformats the time. As mentioned in the doc string some data isn’t in the dictionary or the final csv file to save space. 

In [ ]:
def NeutronDataTranslator(Neutron_key):
    "Takes a Neutron_key and returns the list of data associated with it. "
    Neutron_file = open(Neutron_folder+"\\"+Neutron_key+".txt", "r")
    contents = Neutron_file.read()
    contents = contents.replace("'","").replace("[","").replace("]","")
    contents = contents.split(", ")
    contents = list(map(float, contents))
    Neutron_file.close()
    return contents

NeutronDataTranslator does as the doc string says. 

In [ ]:
def NewRun(run_number, run_data, time_list, raw_data, file_location):
    """This activates if a new run has started, it makes a new list of dictionaries and makes a CSV out of current data"""
    if type(time_list) == int:
        df1 = pd.DataFrame()
        df1["Date/Time"] = raw_data["neutron_start_time"]
        for i in raw_data:
            df1[str(i)] = str(raw_data[i])
        print(run_number + " in " + file_location)
        df1.to_csv(file_location+"\\"+run_number)
    else:
        df1 = pd.DataFrame()
        for i in time_list:
            row = np.array([i])
            column_names = np.array(["Date/Time"])
            for j in run_data:
                try:
                    row = np.append(row, [j[1][i]])
                    column_names = np.append(column_names, [j[0]])
                except:
                    pass
            try:
                df2 = pd.DataFrame([row], columns = column_names)
                df1 = df1.append(df2, sort=False)
            except:
                df2 = pd.DataFrame([row][:-1], columns = column_names)
                df1 = df1.append(df2, sort=False)
        for i in raw_data:
            df1[str(i)] = str(raw_data[i])
        print(run_number + " in " + file_location)
        df1.to_csv(file_location+"\\"+run_number)
    
    raw_data = {}
    run_data = []
    time_list = np.array([])
    for i,j in enumerate(col_names):
        run_data.append((j,{}))
    
    return run_data,time_list,raw_data

NewRun is the function that resets all the locally remembered variables for each run and makes the file for the run that has just finished being processed. It does this by making a pandas data frame that is empty apart from the column name of Date/Time, from there data from the txt files are loaded into a row of data, put into a second data frame, the second data frame is appended to the first and this iterates until all the run data is in one data frame. The raw data is then added to the data frame as it is constant a column is initiated with the  default value from the raw data values and column names from the raw data keys.

In [ ]:
def AddData(file_path, filename, n, run_data, time_list):
    """This takes data from a txt file and puts it into a list of times and the dictionaries"""
    df = pd.read_csv(file_path, sep = "\t", names = ["Date/Time", str(true_name_translater[filename[n:]])])
    file_times = df["Date/Time"].values
    file_data = df[str(true_name_translater[filename[n:]])].values 
    dict_index = Find_dict(run_data, true_name_translater[filename[n:]])
    if os.stat(file_path).st_size == 0: # remove empty files
        pass
    elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
        pass
    elif file_times[0][10] =="T":
        for i in range(len(li)): # fix the T issue in some times 
            untdate = file_times[0].replace('T',' ')
            file_times = np.append(file_times,[untdate])
            file_times = np.delete(file_times,0, axis = None)
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:#Alot of tempreture files, may be some overlap in time
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:#rest of the files 
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    else: # Normal time columns
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    return run_data, time_list


AddData is the function that takes the data from the txt file and puts it into run_data, it uses a lot of the things made and explained previously. It also uses another function Find_dict.

In [ ]:
def Find_dict(run_data, trans_filename):
    """Takes the run_data dictionary and the universal file name (called trans_filename as true name translator 
    must be used) and returns the index the universal file name occurs in the run_data. """
    for i,j in enumerate(run_data):
        if j[0] == trans_filename:
            return i

Does what the doc string says.

In [ ]:
#%%timeit
#%prun
directory = r"C:\Users\zahaab\test_case4"
output = directory + "\\temp_raw_data"

Neutron_folder = CreateFolder(directory+"\\"+"Neutron_data")
ENGINX_DATA = CreateFolder(directory+"\\"+"clean_data")
data = ["North_neutron_X_data","North_neutron_Y_data","South_neutron_X_data",
            "South_neutron_Y_data","neutron_start_time","neutron_end_time","Run_Title"]

for foldername in os.listdir(directory):
    print(foldername)
    if foldername[0:5] == 'cycle':
        folder_path = directory+"\\"+foldername
        clean_data = CreateFolder(ENGINX_DATA+"\\"+foldername)
        n = FindDel(os.listdir(folder_path))
        filenames = os.listdir(folder_path)
        raw_data = {}
        run_data = []
        for i,j in enumerate(col_names):
            run_data.append((j,{}))
        time_list = np.array([])
        finale_file = 0
        run_index = RunIndex(filenames[0],n)
        
        for filename in reversed(filenames):  
            if filename[0:3] == 'ENG':
                finale_file = filename
                break
        
        for k, filename in enumerate(filenames):
            print(filename)
            if k == 0:
                continue
                
            elif "Copy" in filename:
                continue
                
            elif "copy" in filename:
                continue
                
            if filename[n] in ("S", "s", "N", "n"):# Stop temporary files from entering (e.g .s01 or .n001)
                try:
                    int(filename[n+1]) # all temp files are 's' or 'n' followed by a int
                    continue
                except ValueError: 
                    pass
            
            print(RunIndex(filenames[k],n))
            print(filename[-3:])
            
            file_path = folder_path+'\\'+filename
            if true_name_translater[filename[n:]] == 0:
                pass
                
            elif filename[-3:] == "nxs":
                pass
            
            elif RunIndex(filenames[k],n) != run_index:
                run_index = RunIndex(filenames[k],n)
                if len(time_list) == 0:
                    pass
                else:
                    run_data,time_list,raw_data = NewRun(RunIndex(filenames[k-1],n), run_data, time_list, raw_data, clean_data)
                    if filename[-3:].lower() in ["RAW", "raw", "Raw"]:
                        raw_data = AddRawData(RunIndex(filenames[k],n))
                    elif filename[-3:] == "txt":
                        run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
                    else:
                        pass
            else:
                if filename[-3:].lower() in ["RAW", "raw", "Raw"]:
                    raw_data = AddRawData(RunIndex(filenames[k],n))
                elif filename[-3:] == "txt":
                    run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
                else:
                    pass
            
            if filename == finale_file:
                break
        NewRun(RunIndex(finale_file,n), run_data, time_list, raw_data, clean_data)

Finally we must run use the functions and this loop finds when each function must be used and uses it.

## Step 5


This step is the most cpu intensive as it involves taking all the data from the newly made directories for each cycle and combines them with all the data from the stress rig for that cycle using the pandas concat function with two large pandas data frames.  

In [ ]:
from datetime import datetime

def CombineRigCleanData(cycle):
    direct = "E:\\stuff\\OtherData\\"+cycle+"\\Stress Rig"
    
    df1 = pd.DataFrame()
    for filename in os.listdir(ENGINX_DATA+"\\"+cycle):
        file_path = ENGINX_DATA+"\\"+cycle+"\\"+filename
        df2 = pd.read_csv(file_path,index_col=0)
        df2["Run_Number"] = filename
        df1 = df1.append(df2, ignore_index=False, sort=False)
    time_list = df1["Date/Time"].values
    df1.set_index("Date/Time") 

    df3 = pd.DataFrame()
    for filename in os.listdir(direct):
        file_path = direct+"\\"+filename
        df4 = pd.read_csv(file_path, sep = "|", skipinitialspace = True, skiprows = [0,1])
        df3 = df3.append(df4, ignore_index=False, sort=False)
        input_file = open(file_path+".txt", "r")
        first_line = output_file.readline()
        constants = {"Cross_Sectional_Area" : first_line[:33].split("=")[-1].replace(" ", ""),
                     "Gauge_Length_for_Strain1" : first_line[33:].split("=")[-1].replace(" ", "")}
        input_file.close()
        for i in constants:
            df3[str(i)] = str(constants[i]).replace("\n","")

    df3 = df3.rename(index = str, columns = dict(zip(
        df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
    df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})

    #df3.set_index("Date/Time")
    #df3 = df3.loc[~df3.index.duplicated(keep='first')] #drop duplicates 
    df3 = df3.loc[~df3["Date/Time"].duplicated(keep='first')]
    df5 = df3.loc[df3["Date/Time"].isin(time_list)]
    result = pd.concat([df5, df1], axis=1, sort=False, join='outer')

    return(result) 

def CleanInstron01(cycle, filenames, clean_file_time):
    direct = r"E:\stuff\OtherData\INSTRON_01"
    
    df1 = pd.DataFrame()
    for filename in os.listdir(ENGINX_DATA+"\\"+cycle):
        file_path = ENGINX_DATA+"\\"+cycle+"\\"+filename
        df2 = pd.read_csv(file_path,index_col=0)
        df2["Run_Number"] = filename
        df1 = df1.append(df2, ignore_index=False, sort=False)
    time_list = df1["Date/Time"].values
    df1.set_index("Date/Time") 
    
    #find the files that needed for the spesific cycle
    cycle_start, cycle_end = time_list[0], time_list[-1]
    start_file_index, end_file_index = 0, 0
    for i in clean_file_time:
        if datetime.strptime(i, "%Y-%m-%d %H:%M:%S") > datetime.strptime(cycle_start, "%Y-%m-%d %H:%M:%S"):
            start_file_index = clean_file_time.index(i)-1
            break 
    for i in clean_file_time.reverse():
        if datetime.strptime(i, "%Y-%m-%d %H:%M:%S") < datetime.strptime(cycle_start, "%Y-%m-%d %H:%M:%S")::
            end_file_index = (clean_file_time.index(i)+1)*-1
            break 
    
    if start_file_index, end_file_index == 0,0:#Logic needs to be fixed
        return(df1)
    
    df3 = pd.DataFrame()
    for filename in os.listdir(direct)[start_file_index, end_file_index+1]:
        file_path = direct+"\\"+filename
        df4 = pd.read_csv(file_path, sep = "\t", skipinitialspace = True, skiprows = [0,1,2,3])
        df3 = df3.append(df4, ignore_index=False, sort=False)
        input_file = open(file_path+".txt", "r")
        first_line, second_line, third_line = output_file.readline(), output_file.readline(1), output_file.readline(2)
        constants = {"Cross_Sectional_Area" : first_line.split("=")[-1].replace(" ", ""),
                     "Gauge_Length_for_Strain1" : second_line.split("=")[-1].replace(" ", ""),
                     "RB Number" : third_line.split("=")[-1].replace(" ", "")}
        input_file.close()
        for i in constants:
            df3[str(i)] = str(constants[i]).replace("\n","")
        
    df3 = df3.rename(index = str, columns = dict(zip(
        df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
    df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})
    df3 = df3.loc[~df3["Date/Time"].duplicated(keep='first')]
    
    unttime = []
    for i in df3["Date/Time"].values:
        untdate = "{}-{}-{} {}:{}:{}".format(
            date[0:4], date[5:7], date[8:10], date[11:13], date[14:16], date[17:19])
        unttime.append(untdate)
    df3 = df3.drop(columns=["Date/Time"])
    df3["Date/Time"] = unttime
    
    df5 = df3.loc[df3["Date/Time"].isin(time_list)]
    result = pd.concat([df5, df1], axis=1, sort=False, join='outer')
    return(result) 

These two functions do the same thing for the most part, the method for storing the stress rig data has changed over time thus both of these functions are needed, they do as the description of step 5 says. Of Course they do a lot of formatting of the data as to make sure to keep all the nomenclature consistent. 

In [ ]:
ENGINX_DATA = r"E:\clean_data"

instron_start = len(os.listdir(ENGINX_DATA))

filenames = os.listdir(r"E:\stuff\OtherData\INSTRON_01")
clean_file_time = []
for filename in filenames:
    if "continuous" in filename:
        filenames.remove(filename)
    else: 
        date = filename[11:]
        untdate = "{}-{}-{} {}:{}:{}".format(
        date[0:4], date[5:7], date[8:10], date[11:13], date[14:16], date[17:19])
        clean_file_time.append(untdate)
            
for i,j in  enumerate(os.listdir(ENGINX_DATA)):
    if j == "cycle_17_3":
        CombineRigCleanData(j)
        instron_start = i
    elif i > instron_start:
        CleanInstron01(j, filenames, clean_file_time)
    else:
        CombineRigCleanData(j)

Here we have to make a list of filenames that are in the instron directory as we must identify the cycle; the cycle isn’t mentioned in the instron data. This is used along with the clean file times to identify the cycle for CleanInstron01.

Finally we must run use the functions and this loop finds when each function must be used and uses it.